# Region of Waterloo Demo, 24 November 2015
This is a notebook to demo how we're forseeing the rapid prototyping of work with web archives.

Note that we can begin to intersperse text with the code that we're writing, to enable the sharing of notebooks and research ideas.

In [ ]:
:cp /Users/ianmilligan1/dropbox/warcbase/target/warcbase-0.1.0-SNAPSHOT-fatjar.jar

globalScope.jars: Array[String] = [Ljava.lang.String;@7ce38a2a
res6: List[String] = List(/Users/ianmilligan1/dropbox/warcbase/target/warcbase-0.1.0-SNAPSHOT-fatjar.jar, file:/Users/ianmilligan1/Dropbox/spark-notebook-0.6.1-scala-2.10.4-spark-1.3.0-hadoop-2.6.0/, file:/Users/ianmilligan1/dropbox/spark-notebook-0.6.1-scala-2.10.4-spark-1.3.0-hadoop-2.6.0/lib/common.common-0.6.1-scala-2.10.4-spark-1.3.0-hadoop-2.6.0.jar)


In [ ]:
import org.warcbase.spark.matchbox._ 
import org.warcbase.spark.rdd.RecordRDD._ 

import org.warcbase.spark.matchbox._
import org.warcbase.spark.rdd.RecordRDD._


In [ ]:
var arc="/Users/ianmilligan1/Dropbox/warcs-workshop/227-20051004191331-00000-crawling015.archive.org.arc.gz";
var warc="/Users/ianmilligan1/dropbox/wahr/sample-data/arc-warc/ARCHIVEIT-227-QUARTERLY-XUGECV-20091218231727-00039-crawling06.us.archive.org-8091.warc.gz";
var arcdir="/Users/ianmilligan1/dropbox/warcs-workshop";

arc: String = /Users/ianmilligan1/Dropbox/warcs-workshop/227-20051004191331-00000-crawling015.archive.org.arc.gz
warc: String = /Users/ianmilligan1/dropbox/wahr/sample-data/arc-warc/ARCHIVEIT-227-QUARTERLY-XUGECV-20091218231727-00039-crawling06.us.archive.org-8091.warc.gz
arcdir: String = /Users/ianmilligan1/dropbox/warcs-workshop


/Users/ianmilligan1/dropbox/warcs-workshop

In [ ]:
val r = 
RecordLoader.loadArc(arc, 
sc) 
.keepValidPages() 
.map(r => ExtractTopLevelDomain(r.getUrl)) 
.countItems() 
.take(10) 

r: Array[(String, Int)] = Array((cpcml.ca,271), (partimarijuana.org,215), (communist-party.ca,156), (westernblockparty.com,144), (liberal.ca,107), (worldsocialism.org,105), (agoracosmopolite.com,103), (wegovern.ca,74), (www.conservative.ca,70), (canadianactionparty.ca,58))


In [ ]:
val r = 
RecordLoader.loadArc(arc,
sc) 
.keepMimeTypes(Set("text/html")) 
.discardDate(null) 
.map(r => { 
val t = ExtractRawText(r.getBodyContent)
NER3Classifier("/Users/ianmilligan1/dropbox/ner/stanford-ner-2015-04-20/classifiers/english.all.3class.distsim.crf.ser.gz")
val entities = NER3Classifier.classify(t)
val len = 100
(r.getCrawldate, r.getMimeType, entities, r.getUrl, if ( t.length > len ) t.substring(0, 
len) else t)})
.collect()

r: Array[(String, String, String, String, String)] = Array((20051004,text/html,{"PERSON":["Bill Pay"],"ORGANIZATION":[],"LOCATION":[]},http://geocities.com/robots.txt,"Yahoo! Yahoo! - Help Sorry, the page you requested was not found. Search Yahoo!    advanced search "), (20051004,text/html,{"PERSON":[],"ORGANIZATION":[],"LOCATION":[]},http://worldsocialism.org/robots.txt,302 Found Found The document has moved here.), (20051004,text/html,{"PERSON":[],"ORGANIZATION":[],"LOCATION":[]},http://partimarijuana.org/robots.txt,404 Not Found Not Found The requested URL /robots.txt was not found on this server.), (20051004,text/html,{"PERSON":[],"ORGANIZATION":[],"LOCATION":[]},http://walnet.org/robots.txt,404 Not Found Not Found The requested URL /robots.txt was not found on this server. Apache/...

In [ ]:
def createClickableLink(url: String, date: String): String = { 
"<a href='http://web.archive.org/web/" + date + "/" + url + "'>" + 
url + "</a>" 
} 


createClickableLink: (url: String, date: String)String


In [ ]:
val r = 
RecordLoader.loadArc(arc, 
sc) 
.keepValidPages() 
.map(r => { 
val t = ExtractRawText(r.getBodyContent) 
val len = 100 
(r.getCrawldate, createClickableLink(r.getUrl, 
r.getCrawldate), if ( t.length > len ) t.substring(0, len) else t)}) 
.collect() 

r: Array[(String, String, String)] = Array((20051004,<a href='http://web.archive.org/web/20051004/http://geocities.com/CapitolHill/2823/'>http://geocities.com/CapitolHill/2823/</a>,"Canadian Clean Start Party Home Page Can You See Yourself As A Member of Parliament? (Why Not Start "), (20051004,<a href='http://web.archive.org/web/20051004/http://walnet.org/ppp/index2.html'>http://walnet.org/ppp/index2.html</a>,PPP: Parti Populaire des Putes Parti Populaire des Putes Coalition Pour les Droits des Travailleuses), (20051004,<a href='http://web.archive.org/web/20051004/http://worldsocialism.org/canada/'>http://worldsocialism.org/canada/</a>,"Socialist Party of Canada - World Socialism - Scientific Socialism / Marxism / Communism / Politics "), (20051004,<a href='http://web.archive.org/web/2...